# Training

### Reward 수렴 판단

In [ ]:
import os

folder = '2024_11_28_13_21' 

# Example usage
log_dir_episode = f'/home/songmu/multipath/rl/results/{folder}/logs/episode'
log_dir_step = f'/home/songmu/multipath/rl/results/{folder}/logs/step'
output_dir = f'output/{folder}'
# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt

def analyze_combined_episode_logs(log_dir, output_dir):
    """
    Analyze combined episode logs and generate graphs for each metric.

    Parameters:
        log_dir (str): Path to the directory containing episode logs.
        output_dir (str): Path to save the output graphs.
    """
    # List all log files in the directory
    log_files = [f for f in os.listdir(log_dir) if f.endswith('.csv')]
    if not log_files:
        print("No log files found in the specified directory.")
        return

    # Combine all logs into a single DataFrame
    combined_data = pd.DataFrame()
    for log_file in log_files:
        log_path = os.path.join(log_dir, log_file)
        try:
            # Read the log file and append to combined DataFrame
            log_data = pd.read_csv(log_path)
            combined_data = pd.concat([combined_data, log_data], ignore_index=True)
        except Exception as e:
            print(f"Error processing file {log_file}: {e}")

    if combined_data.empty:
        print("No data found in the logs.")
        return

    # Ensure the data is sorted by episode
    combined_data.sort_values(by='Episode', inplace=True)

    # Plot Total Reward per Episode
    plt.figure(figsize=(10, 6))
    plt.plot(combined_data['Episode'], combined_data['Return'], marker='o', markersize = 2, label='Return')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title('Total Reward per Episode')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, 'total_reward_per_episode.png'))
    plt.close()

    # Plot Episode Length per Episode
    plt.figure(figsize=(10, 6))
    plt.plot(combined_data['Episode'], combined_data['Episode Length'], marker='o', markersize = 2, label='Episode Length')
    plt.xlabel('Episode')
    plt.ylabel('Episode Length')
    plt.title('Episode Length per Episode')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, 'episode_length_per_episode.png'))
    plt.close()

    # Plot Average Reward per Episode
    plt.figure(figsize=(10, 6))
    plt.plot(combined_data['Episode'], combined_data['Average Reward (100 eps)'], marker='o', markersize = 2, label='Average Reward')
    plt.xlabel('Episode')
    plt.ylabel('Average Reward')
    plt.title('Average Reward per Episode')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, 'average_reward_per_episode.png'))
    plt.close()

    # Plot Average Loss per Episode
    plt.figure(figsize=(10, 6))
    plt.plot(combined_data['Episode'], combined_data['Average Loss'], marker='o', markersize = 2, label='Average Loss')
    plt.xlabel('Episode')
    plt.ylabel('Average Loss')
    plt.title('Average Loss per Episode')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, 'average_loss_per_episode.png'))
    plt.close()

    # Plot Learning rate
    plt.figure(figsize=(10, 6))
    plt.plot(combined_data['Episode'], combined_data['Learning Rate'], marker='o', markersize = 2, label='Learning Rate')
    plt.xlabel('Episode')
    plt.ylabel('Learning Rate')
    plt.title('Learning Rate')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, 'learning_rate.png'))
    plt.close()

    print(f"Graphs saved in {output_dir}")

analyze_combined_episode_logs(log_dir_episode, output_dir)


KeyError: 'Learning Rate'

<Figure size 1000x600 with 0 Axes>

In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt

def load_filtered_logs(log_dir_step, episode_range=None):
    """
    지정된 범위의 에피소드 데이터만 로드하는 함수.
    
    Args:
        log_dir_step (str): Step 로그 파일들이 있는 디렉토리 경로.
        episode_range (list or range): 로드할 에피소드 번호 리스트 또는 범위.
        
    Returns:
        pd.DataFrame: 필터링된 데이터프레임.
    """
    log_files = sorted([os.path.join(log_dir_step, f) for f in os.listdir(log_dir_step) if f.endswith('.csv')])

    if not log_files:
        print("No log files found in the directory!")
        return None

    filtered_logs = []
    for file in log_files:
        # 데이터 로드
        df = pd.read_csv(file)

        # 지정된 에피소드 범위만 필터링
        if episode_range:
            df = df[df['Episode'].isin(episode_range)]

        if not df.empty:
            filtered_logs.append(df)

    if not filtered_logs:
        print("No logs matched the specified episode range.")
        return None

    # 필터링된 데이터를 병합
    return pd.concat(filtered_logs, ignore_index=True)

def plot_action_over_steps(filtered_logs, output_dir):
    """
    Step별 Action 선택을 시각화하는 함수.
    
    Args:
        filtered_logs (pd.DataFrame): 필터링된 로그 데이터.
        output_dir (str): 그래프 저장 디렉토리.
    """
    plt.figure(figsize=(10, 6))
    
    for episode in filtered_logs['Episode'].unique():
        episode_data = filtered_logs[filtered_logs['Episode'] == episode]
        plt.plot(episode_data['Step'], episode_data['Action'], label=f"Episode {episode}", alpha=0.7, linestyle='none', marker='o', markersize=0.7)
    
    plt.title("Action Selection over Steps")
    plt.xlabel("Step")
    plt.ylabel("Action")
    plt.legend()
    plt.grid(alpha=0.7)
    
    # 그래프 저장
    output_path = os.path.join(output_dir, 'action_over_steps.png')
    plt.savefig(output_path)
    plt.close()
    print(f"Action over Steps plot saved to: {output_path}")


def plot_action_distribution(filtered_logs, output_dir):
    """
    Action 분포를 시각화하는 함수.
    
    Args:
        filtered_logs (pd.DataFrame): 필터링된 로그 데이터.
        output_dir (str): 그래프 저장 디렉토리.
    """
    # Action 분포 시각화
    action_counts = filtered_logs['Action'].value_counts().sort_index()

    plt.figure(figsize=(8, 6))
    action_counts.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title('Action Distribution')
    plt.xlabel('Action')
    plt.ylabel('Frequency')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # 그래프 저장
    output_path = os.path.join(output_dir, 'action_distribution.png')
    plt.savefig(output_path)
    plt.close()
    print(f"Action distribution plot saved to: {output_path}")

def plot_metrics_over_steps(filtered_logs, output_dir):
    """
    Step별 주요 메트릭 시각화 (SSIM, Data Size, Reward).
    
    Args:
        filtered_logs (pd.DataFrame): 필터링된 로그 데이터.
        output_dir (str): 그래프 저장 디렉토리.
    """
    metrics = ['SSIM', 'Data Size', 'Reward']
    for metric in metrics:
        plt.figure(figsize=(8, 6))
        for episode in filtered_logs['Episode'].unique():
            episode_data = filtered_logs[filtered_logs['Episode'] == episode]
            plt.plot(episode_data['Step'], episode_data[metric], label=f"Episode {episode}", linestyle='none', marker='o', markersize=0.7)
        
        plt.title(f'{metric} over Steps')
        plt.xlabel('Step')
        plt.ylabel(metric)
        plt.legend()
        plt.grid(alpha=0.7)

        # 그래프 저장
        output_path = os.path.join(output_dir, f'{metric.lower()}_over_steps.png')
        plt.savefig(output_path)
        plt.close()
        print(f"{metric} over Steps plot saved to: {output_path}")

# Example usage 
# log_dir_step = '/path/to/logs/step'
# output_dir = '/path/to/output'
# os.makedirs(output_dir, exist_ok=True)

# 원하는 에피소드 범위 설정
episode_range = range(450, 451)

# 로그 데이터 로드
filtered_logs = load_filtered_logs(log_dir_step, episode_range)

if filtered_logs is not None:
    # Action 분포 시각화
    plot_action_distribution(filtered_logs, output_dir)
    # Step별 메트릭 시각화
    plot_metrics_over_steps(filtered_logs, output_dir)
    # Step 별 Action
    plot_action_over_steps(filtered_logs, output_dir)


Action distribution plot saved to: output/2024_11_28_13_21/action_distribution.png
SSIM over Steps plot saved to: output/2024_11_28_13_21/ssim_over_steps.png
Data Size over Steps plot saved to: output/2024_11_28_13_21/data size_over_steps.png
Reward over Steps plot saved to: output/2024_11_28_13_21/reward_over_steps.png
Action over Steps plot saved to: output/2024_11_28_13_21/action_over_steps.png
